#### Import packages

In [1]:
import numpy as np
import  pandas as pd
import tensorflow as tf

#### About Dataset

The effectiveness of cancer prediction system helps the people to know their cancer risk with low cost and it also helps the people to take the appropriate decision based on their cancer risk status. The data is collected from the website online lung cancer prediction system .

Total no. of attributes:16 \
No .of instances:284 \
Attribute information:

Gender: M(male), F(female) \
Age: Age of the patient \
Smoking: YES=2 , NO=1. \
Yellow fingers: YES=2 , NO=1. \
Anxiety: YES=2 , NO=1. \
Peer_pressure: YES=2 , NO=1. \
Chronic Disease: YES=2 , NO=1. \
Fatigue: YES=2 , NO=1. \
Allergy: YES=2 , NO=1. \
Wheezing: YES=2 , NO=1. \
Alcohol: YES=2 , NO=1. \
Coughing: YES=2 , NO=1. \
Shortness of Breath: YES=2 , NO=1. \
Swallowing Difficulty: YES=2 , NO=1. \
Chest pain: YES=2 , NO=1. \
Lung Cancer: YES , NO.

#### Import data

In [21]:
data = pd.read_csv("data/lung cancer survey.csv")
print("csv of ", len(data) , "instances")
print("data columns: ", data.columns.to_list())

csv of  309 instances
data columns:  ['GENDER', 'AGE', 'SMOKING', 'YELLOW_FINGERS', 'ANXIETY', 'PEER_PRESSURE', 'CHRONIC DISEASE', 'FATIGUE ', 'ALLERGY ', 'WHEEZING', 'ALCOHOL CONSUMING', 'COUGHING', 'SHORTNESS OF BREATH', 'SWALLOWING DIFFICULTY', 'CHEST PAIN', 'LUNG_CANCER']


In [41]:
X_train = data.iloc[:, :-1] #Get all columns except last
print("size of Xtrain df is:", X_train.shape)
X_train.head()

size of Xtrain df is: (309, 15)


,GENDER,AGE,SMOKING,YELLOW_FINGERS,ANXIETY,PEER_PRESSURE,CHRONIC DISEASE,FATIGUE,ALLERGY,WHEEZING,ALCOHOL CONSUMING,COUGHING,SHORTNESS OF BREATH,SWALLOWING DIFFICULTY,CHEST PAIN
0,M,69,1,2,2,1,1,2,1,2,2,2,2,2,2
1,M,74,2,1,1,1,2,2,2,1,1,1,2,2,2
2,F,59,1,1,1,2,1,2,1,2,1,2,2,1,2
3,M,63,2,2,2,1,1,1,1,1,2,1,1,2,2
4,F,63,1,2,1,1,1,1,1,2,1,2,2,1,1


In [42]:
y_train = data.iloc[:, -1:] #Get only last column
print("Size of y_train df is: ",y_train.shape)
y_train.head()

Size of y_train df is:  (309, 1)


,LUNG_CANCER
0,YES
1,YES
2,NO
3,NO
4,NO


#### Create tf.dataset & pre-proccessing

In [176]:
def preprocess(features, labels):
    features['GENDER'] = tf.where(  #Convert GENDER F/M to 1/2
        tf.equal(features['GENDER'], tf.constant('F', dtype=tf.string)),
        tf.constant(1, dtype=tf.int64),
        tf.constant(2, dtype=tf.int64)
    )
    
    labels = tf.where(      #Convert labels No/Yes 1/2
        tf.equal(labels, tf.constant('NO', dtype=tf.string)),
        tf.constant(0, dtype=tf.int64),
        tf.constant(1, dtype=tf.int64)
    )

    feature_values = tf.stack([features['GENDER'], features['AGE'], features['SMOKING'],
                           features['YELLOW_FINGERS'], features['ANXIETY'], 
                           features['PEER_PRESSURE'], features['CHRONIC DISEASE'],
                           features['FATIGUE '], features['ALLERGY '], features['WHEEZING'],
                           features['ALCOHOL CONSUMING'], features['COUGHING'],
                           features['SHORTNESS OF BREATH'], features['SWALLOWING DIFFICULTY'],
                           features['CHEST PAIN']], axis=-1)
    return feature_values, labels

In [223]:
batch_size = 10
features_ds = tf.data.Dataset.from_tensor_slices(dict(X_train))
labels_ds = tf.data.Dataset.from_tensor_slices(y_train)

dataset = tf.data.Dataset.zip((features_ds, labels_ds))
dataset = dataset.map(preprocess)
dataset = dataset.batch(batch_size)
dataset = dataset.shuffle(buffer_size=X_train.shape[0])
dataset = dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
print("Current dataset: ", dataset)
for features, labels in dataset.take(1):
    tf.print("Features:", features)
    tf.print("Labels:", labels)

Current dataset:  <_PrefetchDataset element_spec=(TensorSpec(shape=(None, 15), dtype=tf.int64, name=None), TensorSpec(shape=(None, 1), dtype=tf.int64, name=None))>
Features: [[2 54 1 ... 1 2 2]
 [1 57 1 ... 1 1 1]
 [2 56 1 ... 1 1 1]
 ...
 [2 64 2 ... 2 1 1]
 [1 61 2 ... 2 2 2]
 [1 72 2 ... 2 2 1]]
Labels: [[1]
 [0]
 [1]
 ...
 [1]
 [1]
 [1]]


#### Split dataset into training & validation

In [224]:
val_ratio = 0.2 #ratio in decimal

total_size = len(list(dataset))
print("Total size in dataset: ", len(list(dataset)))
train_size = int((1 - val_ratio) * total_size)
val_size = total_size - train_size

# Create training and validation datasets
train_dataset = dataset.take(train_size)
val_dataset = dataset.skip(train_size).take(val_size)
print("Train dataset count: ", len(list(train_dataset)))
print("Validation dataset count: ", len(list(val_dataset)))

Total size in dataset:  31
Train dataset count:  24
Validation dataset count:  7


#### Model creation

In [225]:
print(X_train.shape[0])
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(X_train.shape[1],), name='Input'),
    #tf.keras.layers.Dense(64, activation='relu', name = 'layer1'),
    tf.keras.layers.Dense(16, activation='relu', name = 'layer2'),
    tf.keras.layers.Dense(8, activation='relu', name = 'layer3'),
    tf.keras.layers.Dense(1,activation='sigmoid', name="Output") #Since it is a binary classification
])

model.summary()

309
Model: "sequential_44"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 layer2 (Dense)              (None, 16)                256       
                                                                 
 layer3 (Dense)              (None, 8)                 136       
                                                                 
 Output (Dense)              (None, 1)                 9         
                                                                 
Total params: 401 (1.57 KB)
Trainable params: 401 (1.57 KB)
Non-trainable params: 0 (0.00 Byte)


_________________________________________________________________


In [226]:
model.compile(optimizer = 'adam', loss = 'mse', metrics=['accuracy','mse'])
model.fit(train_dataset, validation_data=val_dataset, epochs=100)

Epoch 1/100


24/24 [==============================] - 1s 9ms/step - loss: 0.5630 - accuracy: 0.2552 - mse: 0.5630 - val_loss: 0.1543 - val_accuracy: 0.8571 - val_mse: 0.1543
Epoch 2/100
24/24 [==============================] - 0s 4ms/step - loss: 0.1230 - accuracy: 0.8577 - mse: 0.1230 - val_loss: 0.1412 - val_accuracy: 0.8406 - val_mse: 0.1412
Epoch 3/100
24/24 [==============================] - 0s 4ms/step - loss: 0.1112 - accuracy: 0.8745 - mse: 0.1112 - val_loss: 0.1517 - val_accuracy: 0.8286 - val_mse: 0.1517
Epoch 4/100
24/24 [==============================] - 0s 4ms/step - loss: 0.1132 - accuracy: 0.8703 - mse: 0.1132 - val_loss: 0.1126 - val_accuracy: 0.8714 - val_mse: 0.1126
Epoch 5/100
24/24 [==============================] - 0s 4ms/step - loss: 0.0898 - accuracy: 0.8996 - mse: 0.0898 - val_loss: 0.0998 - val_accuracy: 0.8857 - val_mse: 0.0998
Epoch 6/100
24/24 [==============================] - 0s 5ms/step - loss: 0.1083 - accuracy: 0.8750 - mse: 0.1083 - val_loss: 0.1989 - val_accuracy: